In [279]:
import pandapower as pp
import pandapower.networks as pn
from pandapower.estimation import estimate, remove_bad_data, chi2_analysis
from pandapower.plotting.plotly import simple_plotly, vlevel_plotly

import pandas as pd

import numpy as np
# from tqdm import tqdm

from scipy import stats
print(pp.__version__)

2.0.1


In [96]:
def create_measurement_unit(df_measurement, net):
    
    list_value=[]
    list_std=[]
    for index, row in df_measurement.iterrows():
        if row['element_type'] =='bus':
            if row['meas_type'] =='v':
                mu= net.res_bus.iloc[row['element'],0]
                sigma = (abs(mu)*upper_bus_accuracy-abs(mu)*lower_bus_accuracy)/4
            elif row['meas_type'] =='p':
                mu= net.res_bus.iloc[row['element'],2]
                sigma = (abs(mu)*upper_bus_accuracy-abs(mu)*lower_bus_accuracy)/4
            elif row['meas_type'] =='q':
                mu= net.res_bus.iloc[row['element'],3]
                sigma = (abs(mu)*upper_bus_accuracy-abs(mu)*lower_bus_accuracy)/4
                
        elif row['element_type'] =='line':
            if row['side']=='from':
                if row['meas_type'] =='p':
                    mu= net.res_line.iloc[row['element'],0]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type'] =='q':
                    mu = net.res_line.iloc[row['element'],1]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type']=='i':
                    mu=net.res_line.iloc[row['element'],6]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
            elif row['side']=='to':
                if row['meas_type'] =='p':
                    mu= net.res_line.iloc[row['element'],2]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type'] =='q':
                    mu = net.res_line.iloc[row['element'],3]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                elif row['meas_type']=='i':
                    mu=net.res_line.iloc[row['element'],7]
                    sigma = (abs(mu)*upper_line_accuracy-abs(mu)*lower_line_accuracy)/4
                    
                    
        elif row['element_type'] =='trafo':
            if row['side']=='from':
                if row['meas_type'] == 'v':
                    mu =net.res_trafo.iloc[row['element'],8]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] == 'p':
                    mu =net.res_trafo.iloc[row['element'],0]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] == 'q':
                    mu =net.res_trafo.iloc[row['element'],1]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
            elif row['side']=='to':
                if row['meas_type'] =='v':
                    mu =net.res_trafo.iloc[row['element'],10]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] =='p':
                    mu =net.res_trafo.iloc[row['element'],2]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
                elif row['meas_type'] =='q':
                    mu =net.res_trafo.iloc[row['element'],3]
                    sigma = (abs(mu)*upper_trafo_accuracy-abs(mu)*lower_trafo_accuracy)/4
#         print(mu)  
        value = np.random.normal(mu, sigma, 1)
        list_value.append(value[0])
        list_std.append(sigma)
    df_measurement['value']=list_value
    df_measurement['std_dev']=list_std

    for index, row in df_measurement.iterrows():
        if row['element_type']=='bus':
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'])
        elif row['element_type']=='line':
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'], side=row['side'])
        elif row['element_type']=='trafo':
            if row['meas_type'] in ['p','q']:
                pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                      std_dev=row['std_dev'], element=row['element'], side=row['side'])
    return df_measurement, net

In [152]:
net=pn.case14()
pp.runpp(net)

In [167]:
print(net)

This pandapower network includes the following parameter tables:
   - bus (14 elements)
   - load (11 elements)
   - gen (4 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (15 elements)
   - trafo (5 elements)
   - measurement (6 elements)
   - poly_cost (5 elements)
   - bus_geodata (14 elements)
 and the following results tables:
   - res_line (15 elements)
   - res_trafo (5 elements)
   - res_ext_grid (1 element)
   - res_load (11 elements)
   - res_shunt (1 element)
   - res_gen (4 elements)
   - res_ext_grid_power_flow (1 element)
   - res_line_power_flow (15 elements)
   - res_load_power_flow (11 elements)
   - res_trafo_power_flow (5 elements)
   - res_shunt_power_flow (1 element)
   - res_gen_power_flow (4 elements)


In [153]:
print(net.line)

    c_nf_per_km   df  from_bus  g_us_per_km  in_service  length_km  \
0    768.484773  1.0         0          0.0        True        1.0   
1    716.088084  1.0         0          0.0        True        1.0   
10     0.000000  1.0         8          0.0        True        1.0   
11     0.000000  1.0         8          0.0        True        1.0   
12     0.000000  1.0         9          0.0        True        1.0   
13     0.000000  1.0        11          0.0        True        1.0   
14     0.000000  1.0        12          0.0        True        1.0   
2    637.493051  1.0         1          0.0        True        1.0   
3    494.857619  1.0         1          0.0        True        1.0   
4    503.590401  1.0         1          0.0        True        1.0   
5    186.299339  1.0         2          0.0        True        1.0   
6      0.000000  1.0         3          0.0        True        1.0   
7      0.000000  1.0         5          0.0        True        1.0   
8      0.000000  1.0

In [154]:
print(net.trafo)

    df  hv_bus  i0_percent  in_service  lv_bus  max_loading_percent  name  \
0  1.0       3         0.0        True       6                100.0  None   
1  1.0       3         0.0        True       8                100.0  None   
2  1.0       4         0.0        True       5                100.0  None   
3  1.0       6         0.0        True       7                100.0  None   
4  1.0       6         0.0        True       8                100.0  None   

   parallel  pfe_kw  shift_degree  ...  tap_min tap_phase_shifter  tap_pos  \
0         1     0.0           0.0  ...      NaN             False     -1.0   
1         1     0.0           0.0  ...      NaN             False     -1.0   
2         1     0.0           0.0  ...      NaN             False     -1.0   
3         1     0.0           0.0  ...      NaN             False      NaN   
4         1     0.0           0.0  ...      NaN             False      NaN   

   tap_side  tap_step_degree  tap_step_percent  vn_hv_kv vn_lv_kv  v

In [166]:
print(net.ext_grid)

   bus  in_service  name  va_degree  vm_pu  max_p_mw  min_p_mw  max_q_mvar  \
0    0        True  None        0.0   1.06     332.4       0.0        10.0   

   min_q_mvar  
0         0.0  


In [170]:
print(net.gen)

   bus  controllable  in_service  name  p_mw  scaling  sn_mva  type  vm_pu  \
0    1          True        True  None  40.0      1.0     NaN  None  1.045   
1    2          True        True  None   0.0      1.0     NaN  None  1.010   
2    5          True        True  None   0.0      1.0     NaN  None  1.070   
3    7          True        True  None   0.0      1.0     NaN  None  1.090   

   slack  max_p_mw  min_p_mw  max_q_mvar  min_q_mvar  
0  False     140.0       0.0        50.0       -40.0  
1  False     100.0       0.0        40.0         0.0  
2  False     100.0       0.0        24.0        -6.0  
3  False     100.0       0.0        24.0        -6.0  


In [164]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['p','q','p','q','p','q']
df_measurement['element_type']=['line','line','line','line','line','line']
df_measurement['element']=[0,0,1,1,14,14]
df_measurement['side']=['from','from','from','from','from','from']

In [165]:
df_measurement, net = create_measurement_unit(df_measurement, net)
#State Estimation 
success = estimate(net, init="flat", calculate_voltage_angles=True, zero_injection=[6])

System is not observable (cancelling)
Measurements available: 2. Measurements required: 27


UserWarning: Measurements available: 2. Measurements required: 27

In [137]:
net.gen

,bus,controllable,in_service,name,p_mw,scaling,sn_mva,type,vm_pu,slack,max_p_mw,min_p_mw,max_q_mvar,min_q_mvar
0,1,True,True,None,40.0,1.0,NaN,None,1.045,False,140.0,0.0,50.0,-40.0
1,2,True,True,None,0.0,1.0,NaN,None,1.010,False,100.0,0.0,40.0,0.0
2,5,True,True,None,0.0,1.0,NaN,None,1.070,False,100.0,0.0,24.0,-6.0
3,7,True,True,None,0.0,1.0,NaN,None,1.090,False,100.0,0.0,24.0,-6.0


In [138]:
net.load

,bus,const_i_percent,const_z_percent,controllable,in_service,name,p_mw,q_mvar,scaling,sn_mva,type
0,1,0.0,0.0,False,True,None,21.7,12.7,1.0,NaN,None
1,2,0.0,0.0,False,True,None,94.2,19.0,1.0,NaN,None
10,13,0.0,0.0,False,True,None,14.9,5.0,1.0,NaN,None
2,3,0.0,0.0,False,True,None,47.8,-3.9,1.0,NaN,None
3,4,0.0,0.0,False,True,None,7.6,1.6,1.0,NaN,None
4,5,0.0,0.0,False,True,None,11.2,7.5,1.0,NaN,None
5,8,0.0,0.0,False,True,None,29.5,16.6,1.0,NaN,None
6,9,0.0,0.0,False,True,None,9.0,5.8,1.0,NaN,None
7,10,0.0,0.0,False,True,None,3.5,1.8,1.0,NaN,None
8,11,0.0,0.0,False,True,None,6.1,1.6,1.0,NaN,None


In [ ]:
net.res_bus-net.res_bus_est

In [ ]:
print(net.res_line-net.res_line_est)

In [33]:
net.trafo

,df,hv_bus,i0_percent,in_service,lv_bus,max_loading_percent,name,parallel,pfe_kw,shift_degree,...,tap_min,tap_phase_shifter,tap_pos,tap_side,tap_step_degree,tap_step_percent,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent
0,1.0,3,0.0,True,6,100.0,None,1,0.0,0.0,...,NaN,False,-1.0,hv,NaN,2.2,135.0,14.000,2070.288,0.0
1,1.0,3,0.0,True,8,100.0,None,1,0.0,0.0,...,NaN,False,-1.0,hv,NaN,3.1,135.0,0.208,5506.182,0.0
2,1.0,4,0.0,True,5,100.0,None,1,0.0,0.0,...,NaN,False,-1.0,hv,NaN,6.8,135.0,0.208,2494.998,0.0
3,1.0,6,0.0,True,7,100.0,None,1,0.0,0.0,...,NaN,False,NaN,None,NaN,NaN,14.0,12.000,1743.885,0.0
4,1.0,6,0.0,True,8,100.0,None,1,0.0,0.0,...,NaN,False,NaN,None,NaN,NaN,14.0,0.208,1089.099,0.0


In [147]:
global upper_bus_accuracy, lower_bus_accuracy, upper_line_accuracy, lower_line_accuracy, upper_trafo_accuracy, lower_trafo_accuracy
upper_bus_accuracy=1.01
lower_bus_accuracy=0.99

upper_line_accuracy=1.03
lower_line_accuracy=0.97

upper_trafo_accuracy=1.01
lower_trafo_accuracy=0.99


net=pn.case14()
pp.runpp(net)


inject_bus=[6]

X=pd.DataFrame()
# create temp tranfo dataframe
# temp_list=[]
# for index, row in net.res_trafo.iterrows():
#     temp_list.append(['v','trafo',index,'to'])
#     temp_list.append(['p','trafo',index,'to'])
#     temp_list.append(['q','trafo',index,'to'])
#     temp_list.append(['v','trafo',index,'from'])
#     temp_list.append(['p','trafo',index,'from'])
#     temp_list.append(['q','trafo',index,'from'])
# temp_df=pd.DataFrame(temp_list)
# # concatrate temp transfo dataframe
# X=pd.concat([X, temp_df])

# create temp line dataframe
temp_list=[]
for index, row in net.res_line.iterrows():
    print(index)
    temp_list.append(['p','line',index,'from'])
    temp_list.append(['q','line',index,'from'])
    temp_list.append(['p','line',index,'to'])
    temp_list.append(['q','line',index,'to'])
    
temp_df=pd.DataFrame(temp_list)
X=pd.concat([X, temp_df])

# create temp line dataframe
temp_list=[]
for index, row in net.res_bus.iterrows():
    temp_list.append(['v','bus',index,'None'])

temp_df=pd.DataFrame(temp_list)
X=pd.concat([X, temp_df])

X.columns=['meas_type','element_type','element','side']
X, net = create_measurement_unit(X, net)



0
1
10
11
12
13
14
2
3
4
5
6
7
8
9


In [149]:
X

,meas_type,element_type,element,side,value,std_dev
0,p,line,0,from,157.582677,2.353243
1,q,line,0,from,-20.439880,0.306064
2,p,line,0,to,-154.380107,2.288779
3,q,line,0,to,27.575530,0.415144
4,p,line,1,from,75.971168,1.132656
5,q,line,1,from,3.846284,0.057825
6,p,line,1,to,-72.464035,1.091213
7,q,line,1,to,2.255279,0.033440
8,p,line,10,from,-22.530402,0.349285
9,q,line,10,from,4.496118,0.067097


In [144]:
net.b.iloc[3,0] in [6]

False

In [151]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.060000,0.000000,-232.393272,16.549301
1,1.045000,-4.982589,-18.300000,-30.857100
10,1.056907,-14.790622,3.500000,1.800000
11,1.055189,-15.075585,6.100000,1.600000
12,1.050382,-15.156276,13.500000,5.800000
13,1.035530,-16.033645,14.900000,5.000000
2,1.010000,-12.725100,94.200000,-6.075348
3,1.017671,-10.312901,47.800000,-3.900000
4,1.019514,-8.773854,7.600000,1.600000
5,1.070000,-14.220946,11.200000,-5.230944


In [131]:
success = estimate(net, init="flat", calculate_voltage_angles=True, zero_injection=[6])

In [132]:
net.res_bus-net.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.138741,0.000000,27.529552,14.664646
1,0.155831,3.510584,-212.131524,-8.037233
2,-0.052501,-15.617462,26.118503,-7.618073
3,-0.093838,-17.559843,39.221400,0.468787
4,-0.080910,-15.841640,-13.229142,6.593884
5,0.013380,-19.677083,15.886844,-24.848364
6,0.169891,-4.922755,1.657508,-0.668926
7,0.200339,-4.824801,-105.069017,-78.398172
8,0.167378,-6.318929,-155.966244,-48.439501
9,-0.068945,-23.961148,243.999687,103.678270


In [124]:
print(net)

This pandapower network includes the following parameter tables:
   - bus (14 elements)
   - load (11 elements)
   - gen (4 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (15 elements)
   - trafo (5 elements)
   - measurement (94 elements)
   - poly_cost (5 elements)
   - bus_geodata (14 elements)
 and the following results tables:
   - res_bus (14 elements)
   - res_line (15 elements)
   - res_trafo (5 elements)
   - res_ext_grid (1 element)
   - res_load (11 elements)
   - res_shunt (1 element)
   - res_gen (4 elements)
   - res_bus_power_flow (14 elements)
   - res_ext_grid_power_flow (1 element)
   - res_line_power_flow (15 elements)
   - res_load_power_flow (11 elements)
   - res_trafo_power_flow (5 elements)
   - res_shunt_power_flow (1 element)
   - res_gen_power_flow (4 elements)
   - res_bus_est (14 elements)
   - res_line_est (15 elements)
   - res_trafo_est (5 elements)


In [125]:
net.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,201.874027,-3.909879,-192.480182,28.302839,9.393845,24.392960,0.941709,0.940060,0.941709,0.916961,0.000000,0.885077,-8.486939,2.224210
1,55.419511,1.953523,-53.435519,2.243608,1.983992,4.197131,0.258635,0.258597,0.258635,0.916961,0.000000,0.884494,-8.611042,0.610866
10,-25.318030,4.878821,25.503070,-4.387279,0.185041,0.491542,66.946501,66.946501,66.946501,1.069044,0.752040,1.072936,1.898301,0.243622
11,-16.741095,14.031371,17.271781,-12.902530,0.530686,1.128841,56.715940,56.715940,56.715940,1.069044,0.752040,1.055113,3.958922,0.206392
12,6.919878,3.626826,-6.876373,-3.524987,0.043505,0.101840,20.211781,20.211781,20.211781,1.072936,1.898301,1.061195,1.379217,0.073552
13,3.928733,1.825860,-3.895090,-1.795421,0.033642,0.030438,10.831824,10.831824,10.831824,1.110175,5.729426,1.099075,5.550093,0.039418
14,12.998844,7.627629,-12.677420,-6.973199,0.321423,0.654430,38.063225,38.063225,38.063225,1.099075,5.550093,1.055113,3.958922,0.138514
2,-0.678506,-2.686813,0.679348,-0.750600,0.000842,-3.437413,0.013390,0.004878,0.013390,0.885077,-8.486939,0.887611,-8.422259,0.031626
3,0.172006,-1.665186,-0.171902,-0.999588,0.000104,-2.664775,0.008089,0.004898,0.008089,0.885077,-8.486939,0.885629,-8.523272,0.019105
4,0.968908,-1.374375,-0.968226,-1.332187,0.000683,-2.706561,0.008125,0.007963,0.008125,0.885077,-8.486939,0.884494,-8.611042,0.019191


In [126]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,156.882891,-20.404292,-152.585290,27.676250,4.297600,7.271958,0.638290,0.634645,0.638290,1.060000,0.000000,1.045000,-4.982589,1.507568
1,75.510382,3.854991,-72.747509,2.229359,2.762873,6.084350,0.305050,0.305305,0.305305,1.060000,0.000000,1.019514,-8.773854,0.721097
10,5.227552,4.219138,-5.214678,-4.184937,0.012875,0.034201,17.658967,17.658967,17.658967,1.055932,-14.938521,1.050985,-15.097288,0.064262
11,9.426381,3.610006,-9.310227,-3.362931,0.116154,0.247075,26.534043,26.534043,26.534043,1.055932,-14.938521,1.035530,-16.033645,0.096559
12,-3.785322,-1.615063,3.797904,1.644514,0.012581,0.029451,10.869239,10.869239,10.869239,1.050985,-15.097288,1.056907,-14.790622,0.039554
13,1.614258,0.753959,-1.607960,-0.748261,0.006298,0.005698,4.686717,4.686717,4.686717,1.055189,-15.075585,1.050382,-15.156276,0.017055
14,5.643851,1.747174,-5.589773,-1.637069,0.054078,0.110105,15.612659,15.612659,15.612659,1.050382,-15.156276,1.035530,-16.033645,0.056815
2,73.237579,3.560203,-70.914310,1.602233,2.323269,5.162436,0.300079,0.300351,0.300351,1.045000,-4.982589,1.010000,-12.725100,0.709395
3,56.131496,-1.550350,-54.454838,3.020687,1.676658,1.470337,0.229806,0.229193,0.229806,1.045000,-4.982589,1.017671,-10.312901,0.542776
4,41.516215,1.170998,-40.612462,-2.099034,0.903753,-0.928036,0.169973,0.170589,0.170589,1.045000,-4.982589,1.019514,-8.773854,0.402912


In [127]:
net.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.916961,0.000000,-2.572935e+02,1.956356
1,0.885077,-8.486939,1.920178e+02,-22.576465
2,1.061195,1.379217,6.787667e+01,1.699076
3,1.110175,5.729426,8.411142e+00,-4.375853
4,1.099075,5.550093,2.060578e+01,-5.071731
5,1.055113,3.958922,-4.594361e+00,19.875729
6,0.887611,-8.422259,-1.731394e+00,0.637859
7,0.885629,-8.523272,1.076302e+02,58.611846
8,0.884494,-8.611042,1.733550e+02,48.014688
9,1.118398,7.330941,-2.225085e+02,-95.604265


## Now i consider on LV system

In [210]:
net=pn.case33bw()
pp.runpp(net)

In [252]:
print(net.line)

    c_nf_per_km   df  from_bus  g_us_per_km  in_service  length_km  max_i_ka  \
0           0.0  1.0         0          0.0        True        1.0   99999.0   
1           0.0  1.0         1          0.0        True        1.0   99999.0   
10          0.0  1.0        10          0.0        True        1.0   99999.0   
11          0.0  1.0        11          0.0        True        1.0   99999.0   
12          0.0  1.0        12          0.0        True        1.0   99999.0   
13          0.0  1.0        13          0.0        True        1.0   99999.0   
14          0.0  1.0        14          0.0        True        1.0   99999.0   
15          0.0  1.0        15          0.0        True        1.0   99999.0   
16          0.0  1.0        16          0.0        True        1.0   99999.0   
17          0.0  1.0         1          0.0        True        1.0   99999.0   
18          0.0  1.0        18          0.0        True        1.0   99999.0   
19          0.0  1.0        19          

In [254]:
for index, row in net.line.iterrows():
    print('index:', index, 'from bus:', net.line.iloc[index,2], 'to bus:', net.line.iloc[index,12])

index: 0 from bus: 0 to bus: 1
index: 1 from bus: 1 to bus: 2
index: 10 from bus: 18 to bus: 19
index: 11 from bus: 19 to bus: 20
index: 12 from bus: 2 to bus: 3
index: 13 from bus: 20 to bus: 21
index: 14 from bus: 2 to bus: 22
index: 15 from bus: 22 to bus: 23
index: 16 from bus: 23 to bus: 24
index: 17 from bus: 5 to bus: 25
index: 18 from bus: 25 to bus: 26
index: 19 from bus: 26 to bus: 27
index: 2 from bus: 10 to bus: 11
index: 20 from bus: 27 to bus: 28
index: 21 from bus: 28 to bus: 29
index: 22 from bus: 29 to bus: 30
index: 23 from bus: 3 to bus: 4
index: 24 from bus: 30 to bus: 31
index: 25 from bus: 31 to bus: 32
index: 26 from bus: 20 to bus: 7
index: 27 from bus: 8 to bus: 14
index: 28 from bus: 11 to bus: 21
index: 29 from bus: 17 to bus: 32
index: 3 from bus: 11 to bus: 12
index: 30 from bus: 24 to bus: 28
index: 31 from bus: 4 to bus: 5
index: 32 from bus: 5 to bus: 6
index: 33 from bus: 6 to bus: 7
index: 34 from bus: 7 to bus: 8
index: 35 from bus: 8 to bus: 9
index:

In [250]:
net.line.iloc[2,2]

10

In [211]:
# df_measurement=pd.DataFrame()
# df_measurement['meas_type']=['p','q','p','q','p','q','p','q','p','q','p','q']
# df_measurement['element_type']=['line']*len(df_measurement['meas_type'])
# df_measurement['element']=[0,0,1,1,14,14,10,10,20,20,36,36]
# df_measurement['side']=['from','from','from','from','from','from']

In [300]:
net=pn.case33bw()
pp.runpp(net)
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['p','q','p','q','p','q']
df_measurement['element_type']=['line', 'line', 'line', 'line','line', 'line']
df_measurement['element']=[9,9,12,12,14,14]
df_measurement['side']=['from','from','from','from','from','from']
print(len(df_measurement))

6


In [301]:
df_measurement, net = create_measurement_unit(df_measurement, net)

print(df_measurement)

  meas_type element_type  element  side     value   std_dev
0         p         line        9  from  0.365711  0.005417
1         q         line        9  from  0.158460  0.002416
2         p         line       12  from  2.392147  0.035443
3         q         line       12  from  1.673263  0.025263
4         p         line       14  from  0.916189  0.014094
5         q         line       14  from  0.452619  0.006859


In [302]:
print(net.line)

    c_nf_per_km   df  from_bus  g_us_per_km  in_service  length_km  max_i_ka  \
0           0.0  1.0         0          0.0        True        1.0   99999.0   
1           0.0  1.0         1          0.0        True        1.0   99999.0   
10          0.0  1.0        10          0.0        True        1.0   99999.0   
11          0.0  1.0        11          0.0        True        1.0   99999.0   
12          0.0  1.0        12          0.0        True        1.0   99999.0   
13          0.0  1.0        13          0.0        True        1.0   99999.0   
14          0.0  1.0        14          0.0        True        1.0   99999.0   
15          0.0  1.0        15          0.0        True        1.0   99999.0   
16          0.0  1.0        16          0.0        True        1.0   99999.0   
17          0.0  1.0         1          0.0        True        1.0   99999.0   
18          0.0  1.0        18          0.0        True        1.0   99999.0   
19          0.0  1.0        19          

In [303]:
df_measurement, net = create_measurement_unit(df_measurement, net)
#State Estimation 
success = estimate(net, init="flat", calculate_voltage_angles=True)

IndexError: index 36 is out of bounds for axis 0 with size 32

In [290]:
net=pn.case33bw()
pp.runpp(net)

success = estimate(net, init="flat", calculate_voltage_angles=True)

ValueError: Cannot set a frame with no defined index and a value that cannot be converted to a Series

In [287]:
p_measurements.side.values.astype

NameError: name 'p_measurements' is not defined